<a href="https://colab.research.google.com/github/pranavgovi/dementia-detection-/blob/main/patch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install patchify

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
projection_dim=64

In [ ]:
from PIL import Image
from numpy import asarray
from patchify import patchify
import cv2
import numpy as np
from patchify import patchify
from PIL import Image
import tensorflow as tf


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
num_classes = 2
input_shape = (280,280,3)

In [ ]:
import glob
Normal = glob.glob('/content/drive/MyDrive/Pitt/Cookie/spectogram/Normal_spec/*.*')
Dimentia = glob.glob('/content/drive/MyDrive/Pitt/Cookie/spectogram/dimentia_spec/*.*')


data = []
labels = []

for i in Normal:   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb', 
    target_size= (280,280))
    image=np.array(image)
    data.append(image)
    labels.append(0)
for i in Dimentia:   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb', 
    target_size= (280,280))
    image=np.array(image)
    data.append(image)
    labels.append(1)


data = np.array(data)
labels = np.array(labels)

from sklearn.model_selection import train_test_split
X_train, X_test, ytrain, ytest = train_test_split(data, labels, test_size=0.2,
                                                random_state=42)

In [ ]:
class Patches(layers.Layer):
    def __init__(self, patch_size):
        super(Patches, self).__init__()
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches

NameError: ignored

In [ ]:
learning_rate = 0.0020
weight_decay = 0.0001
batch_size = 256
num_epochs = 50
image_size = 72  # We'll resize input images to this size
patch_size = 6  # Size of the patches to be extract from the input images
num_patches = (image_size // patch_size) ** 2
projection_dim = 64
num_heads = 4
transformer_units = [
    projection_dim * 2,
    projection_dim ,
]  # Size of the transformer layers
transformer_layers = 8
mlp_head_units = [2048, 1024] 

In [ ]:
def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa
data_augmentation = keras.Sequential(
    [
        layers.Normalization(),
        layers.Resizing(72, 72),
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(factor=0.02),
        layers.RandomZoom(
            height_factor=0.2, width_factor=0.2
        ),
    ],
    name="data_augmentation",
)
data_augmentation.layers[0].adapt(X_train)

ModuleNotFoundError: ignored

In [ ]:
class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super(PatchEncoder, self).__init__()
        self.num_patches = num_patches
        self.projection = layers.Dense(units=projection_dim)
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )
 
    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded

In [ ]:
def create_vit_classifier():
    inputs = layers.Input(shape=input_shape)
    # Augment data.
    augmented = data_augmentation(inputs)
    # Create patches.
    patches = Patches(patch_size)(augmented)
    # Encode patches.
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)


    # Create multiple layers of the Transformer block.
    for _ in range(transformer_layers):
        # Layer normalization 1.
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        # Create a multi-head attention layer.
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=0.1
        )(x1, x1)
        # Skip connection 1.
        x2 = layers.Add()([attention_output, encoded_patches])
        # Layer normalization 2.
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        # MLP.
        x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.1)
        # Skip connection 2.
        encoded_patches = layers.Add()([x3, x2])

    # Create a [batch_size, projection_dim] tensor.
    representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    representation = layers.Flatten()(representation)
    representation = layers.Dropout(0.5)(representation)
    # Add MLP.
    features = mlp(representation, hidden_units=mlp_head_units, dropout_rate=0.5)
    # Classify outputs.
    logits = layers.Dense(num_classes)(features)
    # Create the Keras model.
    model = keras.Model(inputs=inputs, outputs=logits)
    return model

In [ ]:
def run_experiment(model):
    optimizer = tfa.optimizers.AdamW(
        learning_rate=learning_rate, weight_decay=weight_decay
    )

    model.compile(
        optimizer=optimizer,
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[
            keras.metrics.SparseCategoricalAccuracy(name="accuracy"),
            keras.metrics.SparseTopKCategoricalAccuracy(5, name="top-5-accuracy"),
      
        ],
    )

    ##checkpoint_filepath = "/tmp/checkpoint"
    ##checkpoint_callback = keras.callbacks.ModelCheckpoint(
        ##checkpoint_filepath,
        ##monitor="val_accuracy",
        ##save_best_only=True,
        ##save_weights_only=True,
    #)

    history = model.fit(
    x=X_train,
    y=ytrain,
    batch_size=batch_size,
    epochs=num_epochs,
    validation_split=0.1,)




vit_classifier = create_vit_classifier()
history = run_experiment(vit_classifier)

Epoch 1/50
1/1 [==============================] - 28s 28s/step - loss: 2.2520 - accuracy: 0.4700 - top-5-accuracy: 1.0000 - val_loss: 65.3624 - val_accuracy: 0.8333 - val_top-5-accuracy: 1.0000
Epoch 2/50
1/1 [==============================] - 6s 6s/step - loss: 60.9673 - accuracy: 0.8000 - top-5-accuracy: 1.0000 - val_loss: 2.5033 - val_accuracy: 0.8333 - val_top-5-accuracy: 1.0000
Epoch 3/50
1/1 [==============================] - 6s 6s/step - loss: 2.6930 - accuracy: 0.8000 - top-5-accuracy: 1.0000 - val_loss: 5.7976 - val_accuracy: 0.3333 - val_top-5-accuracy: 1.0000
Epoch 4/50
1/1 [==============================] - 6s 6s/step - loss: 7.2784 - accuracy: 0.7100 - top-5-accuracy: 1.0000 - val_loss: 34.0538 - val_accuracy: 0.1667 - val_top-5-accuracy: 1.0000
Epoch 5/50
1/1 [==============================] - 6s 6s/step - loss: 34.5771 - accuracy: 0.2000 - top-5-accuracy: 1.0000 - val_loss: 4.9769 - val_accuracy: 0.1667 - val_top-5-accuracy: 1.0000
Epoch 6/50
1/1 [=======================

In [ ]:
 _, accuracy, top_5_accuracy = vit_classifier.evaluate(X_test, ytest)
print(f"Test accuracy: {round(accuracy * 100, 2)}%")
print(f"Test top 5 accuracy: {round(top_5_accuracy * 100, 2)}%")



1/1 [==============================] - 1s 610ms/step - loss: 0.5209 - accuracy: 0.8571 - top-5-accuracy: 1.0000
Test accuracy: 85.71%
Test top 5 accuracy: 100.0%


In [ ]:
pip install tensorflow_addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 27.3 MB/s 


In [ ]:
from sklearn.metrics import accuracy_score
p_pred = vit_classifier.predict(X_test)
# [1. 0.01 0.91 0.87 0.06 0.95 0.24 0.58 0.78 ...

# extract the predicted class labels
import numpy as np
rounded_labels=np.argmax(p_pred, axis=1)
print(accuracy_score(ytest, rounded_labels))

# [1 0 1 1 0 1 0 1 1 0 0 0 0 1 1 0 1 0 0 0 0 ...
print(confusion_matrix(ytest, y_pred))
# [[13  1]
#  [ 2  9]]

##(classification_report(ytest, y_pred))

0.8571428571428571


In [ ]:
from sklearn import metrics
metrics.f1_score(ytest, rounded_labels)

0.923076923076923

In [ ]:
from sklearn import metrics
metrics.recall_score(ytest,rounded_labels)

1.0

In [ ]:
from sklearn import metrics
metrics.precision_score(ytest,rounded_labels)

0.8571428571428571

In [ ]:
vit_classifier.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 280, 280, 3  0           []                               
                                )]                                                                
                                                                                                  
 data_augmentation (Sequential)  (None, 72, 72, 3)   7           ['input_2[0][0]']                
                                                                                                  
 patches_1 (Patches)            (None, None, 108)    0           ['data_augmentation[0][0]']      
                                                                                                  
 patch_encoder_1 (PatchEncoder)  (None, 144, 64)     16192       ['patches_1[0][0]']        